In [ ]:
!cp -r ../input/swin-mmdetection/* ./
!cp ../input/swin-weights/swin_base_patch4_window12_384_22k.pth ./

!pip install ./src/mmdet-2.11.0/mmdet-2.11.0/
!pip install ./src/mmpycocotools-12.0.3/mmpycocotools-12.0.3/
!pip install ./src/addict-2.4.0-py3-none-any.whl
!pip install ./src/yapf-0.30.0-py2.py3-none-any.whl
!pip install ./src/mmcv_full-1.2.6-cp37-cp37m-manylinux1_x86_64.whl
!pip install ./src/timm-0.4.5-py3-none-any.whl

In [ ]:
import os
import json
import pickle
import zlib
import base64
import typing as t
import numpy as np
import pandas as pd

# PyTorch
import torch

# MMdetection
import mmcv
from mmcv import Config
from mmcv.parallel import MMDataParallel
from mmdet.datasets import build_dataloader, build_dataset
from mmdet.apis import single_gpu_test
from mmdet.models import build_detector

# COCO
from pycocotools import mask as mutils
from pycocotools import _mask as coco_mask

### I - Helpers

In [ ]:
# Helpers

def get_testing_annotations(df,
                            image_id='ID',
                            image_width='ImageWidth',
                            image_height='ImageHeight',
                            filename='test_annotations.json'):
    custom_annotations = []
    for idx, row in df.iterrows():
        custom_annotation = {'filename': f'{row[image_id]}_green.png',
                             'width': row[image_width],
                             'height': row[image_height],
                             'ann': {'bboxes': None, 'labels': None, 'masks': None}
                            }
        custom_annotations.append(custom_annotation)
    with open(filename, 'w') as json_file:
        json.dump(custom_annotations, json_file)
    return custom_annotations


def encode_binary_mask(mask: np.ndarray) -> t.Text:
    """Converts a binary mask into OID challenge encoding ascii text."""
    # check input mask --
    if mask.dtype != np.bool:
        mask_dtype_error = "encode_binary_mask expects a binary mask"
        raise ValueError(f"{mask_dtype_error}, received dtype == {mask.dtype}")
    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        mask_shape_error = "encode_binary_mask expects a 2d mask"
        raise ValueError(f"{mask_shape_error}, received shape == {mask.dtype}")
    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)
    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]
    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str


### II - Inference

In [ ]:
df = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
# Build test annotations
custom_test_annotations = get_testing_annotations(df)

# Build model config
config_file = '../input/swin-mmdetection/configs/htc/htc+_swin_base_patch4_window12_1x_coco.py'
cfg = Config.fromfile(config_file)
cfg.data.samples_per_gpu = 1

# Build testing dataset
test_dataset = build_dataset(cfg.data.test)
test_data_loader = build_dataloader(test_dataset,
                                    cfg.data.samples_per_gpu,
                                    cfg.data.workers_per_gpu,
                                    # cfg.gpus will be ignored if distributed
                                    # len(cfg.gpu_ids),
                                    dist=False,
                                    shuffle=False)
# Build the detector
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
model.CLASSES = test_dataset.CLASSES
model = MMDataParallel(model, device_ids=[0])
results = single_gpu_test(model, test_data_loader, show=False, out_dir=None, show_score_thr=0.3)

In [ ]:
!rm -rf ./*

### III - Generate submission file

In [ ]:
%%time

with open('submission.csv', 'w') as submission_file:
    print('ID,ImageWidth,ImageHeight,PredictionString', file=submission_file)
    for i in range(len(custom_test_annotations)):
        img_id = custom_test_annotations[i]['filename'].replace('.png', '')
        prediction_strings = []
        for class_id in range(19):
            bounding_boxes = results[i][0][class_id]
            segmentations = results[i][1][class_id]
            for bounding_box, segmentation in zip(bounding_boxes, segmentations):
                box, cnf = bounding_box[:4], bounding_box[4]
                h, w = segmentation['size']
                # convert coco format to kaggle format
                mask = mutils.decode(segmentation).astype(bool)
                rle = encode_binary_mask(mask).decode('ascii')
                prediction_strings.append(f'{class_id} {cnf} {rle}')
        print(f'{img_id},{w},{h},{" ".join(prediction_strings)}', file=submission_file)